## **ACS Explorer** ##

-**Description**: The purpose of this notebook is to provide standard queries for pulling data from ACS microdata related to SNAP enrolled households. With microdata, you can create custom tabulations, but be wary of sample sample sizes. Make sure to use household or person weights. 

-**Date Created**: 10/21/2025




**Libraries**

In [ ]:
import pandas as pd
import requests
import json
import numpy as np

**API Key**

Get API key here: https://api.census.gov/data/key_signup.html

In [ ]:
access_token = notebookutils.credentials.getSecret('https://keyvaultddp.vault.azure.net/', 'census-api-key')

**Get Microdata - Households with POVPIP AND FSTAP**

Full list of variables available here -- https://api.census.gov/data/2022/acs/acs1/pumspr/variables.html

PUMA crosswalk here -- https://www2.census.gov/geo/maps/DC2020/PUMA/st06_ca/Catalog_PUMAmaps_st06.pdf



In [ ]:
variables = [
    'SERIALNO',  # household ID
    'SPORDER',   # person number within household
    'AGEP',      # age of person
    'RAC1P',     # race code
    'HISP',      # Hispanic origin
    'NOC',       # number of own children in household
    'FS',        # food stamps / SNAP recipiency (household-level)
    'WGTP',      # household weight
    'PWGTP',     # person weight
    'COW',       # class of worker (1–8; 5 = federal government)
    'PUMA',       # Public Use Microdata Area
    'POVPIP'      #poverty rate
]


# --- California PUMAs (SF + nearby) ---
pumas = [
    "04103", "04104",  # Marin County (north of SF)
    "07507", "07508", "07509", "07510", "07511", "07512", "07513", "07514"  # San Francisco County
]

state = "06"  # California FIPS

# Base URL for ACS 1-year PUMS
base_url = "https://api.census.gov/data/2023/acs/acs1/pums"

# --- Fetch loop ---
all_data = []

for puma in pumas:
    request_url = (
        f"{base_url}?get={','.join(variables)}"
        f"&for=public use microdata area:{puma}"
        f"&in=state:{state}"
        f"&key={access_token}"
    )

    print(f"Fetching PUMA {puma} ...")

    r = requests.get(request_url)

    if r.status_code == 200:
        census = json.loads(r.text)
        headers = census[0]
        rows = census[1:]
        df = pd.DataFrame(rows, columns=headers)
        df["PUMA"] = puma
        all_data.append(df)
    else:
        print(f"⚠️ Failed to fetch PUMA {puma}: HTTP {r.status_code}")

# --- Combine all results ---
if all_data:
    df = pd.concat(all_data, ignore_index=True)

    # Convert numeric columns where relevant
    num_cols = ['AGEP', 'RAC1P', 'HISP', 'NOC', 'FS', 'WGTP', 'PWGTP', 'COW']
    df[num_cols] = df[num_cols].apply(pd.to_numeric, errors='coerce')

    print(df.head())
    print(f"\n✅ Retrieved {len(df):,} rows across {len(pumas)} PUMAs.")
else:
    print("❌ No data retrieved.")

Fetching PUMA 04103 ...
Fetching PUMA 04104 ...
Fetching PUMA 07507 ...
Fetching PUMA 07508 ...
Fetching PUMA 07509 ...
Fetching PUMA 07510 ...
Fetching PUMA 07511 ...
Fetching PUMA 07512 ...
Fetching PUMA 07513 ...
Fetching PUMA 07514 ...
        SERIALNO SPORDER  AGEP  RAC1P  HISP  NOC  FS  WGTP  PWGTP  COW   PUMA  \
0  2023HU0435650       1    53      1     1    0   2    64     64    5  04103   
1  2023HU0435650       2    51      1     5    0   2    64     55    0  04103   
2  2023HU0444186       1    86      1     1    0   2   122    122    4  04103   
3  2023HU0445113       1    62      1     1    0   2    54     54    1  04103   
4  2023HU0445113       2    62      1     1    0   2    54     61    1  04103   

  POVPIP state public use microdata area  
0    501    06                     04103  
1    501    06                     04103  
2    501    06                     04103  
3    501    06                     04103  
4    501    06                     04103  

✅ Retrieved 11

**Relabel for Clarity**

In [ ]:
#variables in scope for relabeling
variables = [
    "RAC1P", "HISP", "FS","COW"
]

#data dictionary
base_url = "https://api.census.gov/data/2023/acs/acs1/pumspr/variables/{}.json"

label_map = {}
value_map = {}

for var in variables:
    response = requests.get(base_url.format(var))
    if response.ok:
        var_json = response.json()
        label_map[var] = var_json.get("label", var)
        values = var_json.get("values", {}).get("item", {})
        if values:
            try:
                value_map[var] = {str(k): v for k, v in values.items()}
            except:
                value_map[var] = values
    else:
        print(f"Warning: Could not fetch metadata for {var}")

# Step 1: Apply value mapping to raw CPS column names
for var in variables:
    print(var)
    if var in df.columns and var in value_map:
        mapping = value_map[var]
        df[var] = df[var].astype(str).map(mapping).fillna(df[var])

# Step 2: Rename columns to readable labels
df.rename(columns=label_map, inplace=True)

df

RAC1P
HISP
FS
COW


,SERIALNO,SPORDER,AGEP,Recoded detailed race code,Recoded detailed Hispanic origin,NOC,Yearly food stamp/Supplemental Nutrition Assistance Program (SNAP) recipiency,WGTP,PWGTP,Class of worker,PUMA,POVPIP,state,public use microdata area
0,2023HU0435650,1,53,White alone,1,0,No,64,64,Federal government employee,04103,501,06,04103
1,2023HU0435650,2,51,White alone,5,0,No,64,55,N/A (less than 16 years old/NILF who last work...,04103,501,06,04103
2,2023HU0444186,1,86,White alone,1,0,No,122,122,State government employee,04103,501,06,04103
3,2023HU0445113,1,62,White alone,1,0,No,54,54,"Employee of a private for-profit co. or bus., ...",04103,501,06,04103
4,2023HU0445113,2,62,White alone,1,0,No,54,61,"Employee of a private for-profit co. or bus., ...",04103,501,06,04103
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11045,2023HU0932032,1,69,White alone,1,0,No,127,128,N/A (less than 16 years old/NILF who last work...,07514,264,06,07514
11046,2023HU0932032,2,69,White alone,1,0,No,127,135,N/A (less than 16 years old/NILF who last work...,07514,264,06,07514
11047,2023HU0936457,1,27,Asian alone,1,0,No,133,132,"Employee of a private for-profit co. or bus., ...",07514,501,06,07514
11048,2023HU0941323,1,83,Asian alone,1,0,Yes,53,54,N/A (less than 16 years old/NILF who last work...,07514,127,06,07514


**How many people live in a household with at least one federal worker?**

In [ ]:
df_snap_hh = df[df['Yearly food stamp/Supplemental Nutrition Assistance Program (SNAP) recipiency']=="Yes"]